In [ ]:
import os
import math
import pymongo
from pymongo import MongoClient
import json
import re, string

data = []
for filename in os.listdir('bbcsport/rugby'):
    if filename.endswith('.txt'):
        with open(os.path.join('bbcsport/rugby', filename)) as f:
            strings = filename.split(".")
            name = "rugby-" + strings[0]
            data_dict = {}
            content = f.read()
            new_content = ''
            for line in content:
                new_content = new_content + re.sub('[^\w\s-]', '', line)
            new_content = re.sub('\n+','\n',new_content)
            new_content = new_content.replace('\n',' ')
            data_dict["filename"] = name
            data_dict["content"] = new_content.lower().strip()
            data.append(data_dict)

In [ ]:
data

In [ ]:
client = MongoClient()
client = MongoClient('localhost', 27017)
db = client.ds8003_final
bbc_sports = db.bbc_sports
result = bbc_sports.insert_many(data)

In [ ]:
math.log(147/40)

In [ ]:
def tf(term,document):
    total_doc_length = len(document)
    count_word = 0
    for word in document:
        if word == term:
            count_word +=1
    return count_word/total_doc_length

def idf(term,documents):
    total_docs = len(documents)
    total_occurences = 0
    for doc in documents:
        words = doc.split(" ")
        for word in words:
            if word == term:
                total_occurences += 1
                break
    return math.log(total_occurences/total_docs)

def build_index(documents):
    index = {}
    doc_vectors = {}
    doc_count = 1
    for doc in documents:
        doc_vectors[doc_count] = {}
        words = doc.split(" ")
        for word in words:
            if word not in doc_vectors[doc_count]:
                tf_val = tf(word,words)
                if word in index:
                    idf_val = index[word]['idf']
                else:
                    index[word] = {}
                    idf_val = idf(word,documents)
                    index[word]['idf'] = idf_val
                score = tf_val*idf_val
                index[word][doc_count] = score
                doc_vectors[doc_count][word] = score
        doc_count += 1 
    return (index,doc_vectors)

In [ ]:
index,doc_vectors = build_index(documents)
index

In [ ]:
doc_vectors

In [ ]:
def euclidean_norm(vector):
    total_sum = 0
    for key,val in vector.items():
        total_sum += val**2
    return math.sqrt(total_sum)
    
def query_vec(query,index):
    terms = query.split(" ")
    query_vec = {}
    for term in terms:
        tf_val = tf(term,terms)
        idf_val = index[term]['idf']
        query_vec[term] = tf_val*idf_val
    return(query_vec)

def calc_sim(query,doc,query_vec,doc_vec,index):
    numerator = 0
    terms = query.split(" ")
    for term in terms:
        if doc in index[term]:
            doc_score = index[term][doc]
        else:
            doc_score = 0
        query_score = query_vec[term]
        numerator += doc_score*query_score
    denominator = euclidean_norm(query_vec)*euclidean_norm(doc_vec)
    return(numerator/denominator)
    
def query(query,index,doc_vectors):
    queryvec = query_vec(query,index)
    scores = []
    for key,val in doc_vectors.items():
        sim = calc_sim(query,key,queryvec,val,index)
        scores.append((key,sim))
    return(scores)

In [ ]:
db.

In [ ]:
query("shoulders comeback",index,doc_vectors)

In [ ]:
query = "Yachvili slotted over four penalties"
query_words = query.split(" ")

In [ ]:
type(query_words[0])